# NPI-9: NPI is not duplicated

Description: check if there is any duplicate NPI. Test will pass if no duclicate appears.

Starting Author: Amy Jin (amy@careset.com)

Date: April 30th, 2018

https://docs.google.com/spreadsheets/d/1IYg01IpssJaWHo6KxO4_dSDgXtYNFy41S5cIHFLvlGQ/edit#gid=604789549

## Connection to Parenthood Server

In [2]:
# Packages import
import os
import sys
import numpy as np
import pandas as pd
from collections import Counter
import operator
import mysql.connector
import sshtunnel
import pureyaml

# Handle path
project_dir = !pwd  # dir of current script/notebook file
config_file = open(project_dir[0] + "/db.yaml");
config = pureyaml.load(config_file.read());

# Argument dictionary for sshtunnel
ssh_config = {
    'ssh_address_or_host': ('parenthood.set.care', 22),
    'ssh_username':        config['ssh_username'],
    'ssh_password':        config['ssh_password'],
    'remote_bind_address': ('127.0.0.1', 3306),
    'local_bind_address':  ('0.0.0.0', 3333),
}

# Argument dictionary for mysql.connector
mysql_config = {
    'user':     config['mysql_user'],
    'password': config['mysql_passwd'],
    'host':     config['mysql_host'],
    'database': 'patch',
    'port':     3333,
}

# Connect to Parenthood server
with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
    print('SSH tunneling successful on port: {}'.format(tunnel.local_bind_port))
    connection = mysql.connector.connect(**mysql_config)
    cur = connection.cursor()
    print('MySQL server connected successfully!')

SSH tunneling successful on port: 3333
MySQL server connected successfully!


## Test Function

In [3]:
# --------------------------------------- Inputs: ---------------------------------------
# 1) db_name:                database name in server
# 2）table_name:             table name
# 3) col_name:               column to test
# --------------------------------------- Outputs: --------------------------------------
# 1) Test result:            PASS/FAIL
# 2) If FAIL, test will print out all duplicate NPIs and their counts.


def npi_9(db_name, table_name, col_name):
    
    with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
        connection = mysql.connector.connect(**mysql_config)
        cur = connection.cursor()
        
        query = ('''
            SELECT *
            FROM {db}.{t1}
            GROUP BY {col1}
            HAVING COUNT({col1}) >1
            LIMIT 1;
        '''.format(db = db_name, t1 = table_name, col1 = col_name))

        cur.execute(query)
        rows = list(sum(cur.fetchall(), ()))

        
        if not len(rows):
            print ("Test result: PASS")
        else:
            print ("Test result: FAIL" + '\n')
            print ("The following {} have duplicates:".format(col_name) + '\n')
            print ("{}, count".format(col_name) + '\n')

            # MySQL query to get duplicate
            query = ('''
                SELECT {col1}, COUNT({col1})
                FROM {db}.{t1}
                GROUP BY {col1}
                HAVING COUNT({col1}) >1;
            '''.format(db = db_name, t1 = table_name, col1 = col_name))

            cur.execute(query)
            
            for row in cur.fetchall():
                for i in range(0,len(row)):
                    print (str(row[i]), end=", ")
                print ('\n')
            
        cur.close()
        connection.close()

## Test Example

In [4]:
 npi_9('_amy', 'test_data_good', 'npi')

Test result: PASS


In [5]:
npi_9('_amy', 'test_data_bad1', 'npi')

Test result: FAIL

The following npi have duplicates:

npi, count

, 96, 

-1001306713, 2, 

-1205461235, 2, 

-1349820946, 2, 

-1365675640, 2, 

-1376151928, 2, 

-1377764121, 2, 

-1410875519, 2, 

-1425303467, 2, 

-1502785851, 2, 

-1510458706, 2, 

-1535025153, 2, 

-1538200748, 2, 

-1562555165, 2, 

-1915662289, 2, 

-1962309126, 2, 

-2054924448, 2, 

-2144945416, 3, 

-2203044989, 2, 

-2224326439, 2, 

-2234220673, 2, 

-2363020625, 3, 

-2387881917, 3, 

-251040449, 2, 

-2555601830, 2, 

-2665407943, 2, 

-2668866588, 3, 

-271287451, 2, 

-2789384489, 2, 

-2916655565, 4, 

-2944881869, 2, 

-2965820727, 2, 

-3111448527, 2, 

-3352465814, 2, 

-3399600624, 2, 

-3415762872, 2, 

-3607377258, 2, 

-3674108797, 2, 

-3754510236, 2, 

-3768680897, 3, 

-3886382645, 2, 

-3890512889, 2, 

-3971089833, 4, 

-3991449917, 2, 

-413127986, 2, 

-4163100764, 2, 

-4246364575, 2, 

-4374609734, 2, 

-4392097471, 2, 

-4429541401, 2, 

-4441901078, 3, 

-4537199178, 2, 

-467453508